In [10]:
import torch
import numpy as np

In [11]:
class_num = 80
anchor_num = 5

In [55]:
def make_layers(cfg, in_channels, batch_norm=True):
    layers = []
    in_channels = in_channels
    for v in cfg:
        if v == 'M':
            layers += [torch.nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d_layer = torch.nn.Conv2d(in_channels=in_channels, out_channels=v[0],
                                           kernel_size=v[1], stride=v[2], padding=v[3], bias=False)
            acvt_layer = torch.nn.LeakyReLU(negative_slope=0.1, inplace=True)
            if batch_norm:
                bn_layer = torch.nn.BatchNorm2d(num_features=v[0])
                layers += [conv2d_layer, bn_layer, acvt_layer]
            else:
                layers += [conv2d_layer, acvt_layer]
            in_channels = v[0]

    return torch.nn.Sequential(*layers)

def init_model_variables(model):
    print("Initialize model's weights ...")
    for m in model.modules():
        if isinstance(m, torch.nn.Conv2d):
            torch.nn.init.kaiming_normal_(tensor=m.weight, mode='fan_out')
            if m.bias is not None:
                torch.nn.init.constant_(tensor=m.bias, val=0.0)
        elif isinstance(m, torch.nn.BatchNorm2d):
            torch.nn.init.constant_(tensor=m.weight, val=1.0)
            torch.nn.init.constant_(tensor=m.bias, val=0.0)
        elif isinstance(m, torch.nn.Linear):
            torch.nn.init.normal_(tensor=m.weight, mean=0.0, std=0.01)
            torch.nn.init.constant_(tensor=m.bias, val=0.0)
            

class DarkNet19(torch.nn.Module):

    def __init__(self):
        super(DarkNet19, self).__init__()
        # [32, 3, 1, 1] represent the parameters in torch.nn.Conv2d:
        # out_channels = 32
        # kernel_size = 3
        # stride = 1
        # padding = 1
        cfg = [[32, 3, 1, 1], 'M',
               [64, 3, 1, 1], 'M',
               [128, 3, 1, 1], [64, 1, 1, 0], [128, 3, 1, 1], 'M',
               [256, 3, 1, 1], [128, 1, 1, 0], [256, 3, 1, 1], 'M',
               [512, 3, 1, 1], [256, 1, 1, 0], [512, 3, 1, 1], [256, 1, 1, 0],
               [512, 3, 1, 1],
               'M', [1024, 3, 1, 1], [512, 1, 1, 0], [1024, 3, 1, 1], [512, 1, 1, 0], [1024, 3, 1, 1]]

        self.front_layer = make_layers(cfg[:16], in_channels=3)
        self.route_layer = make_layers(cfg[16:17], in_channels=256)
        self.rear_layer = make_layers(cfg[17:], in_channels=512)

    def forward(self, x):
        feature = self.front_layer(x)
        route = self.route_layer(feature)
        feature = self.rear_layer(route)

        return route, feature


class Backbone(torch.nn.Module):
    def __init__(self):
        super(Backbone, self).__init__()
        self.darknet19 = DarkNet19()

        total_bbox_num = (5 + class_num) * anchor_num
        yolov2_cfg = [[64, 1, 1, 0], [256, 3, 2, 1], [1024, 3, 1, 1], [total_bbox_num, 1, 1, 0]]
        # passthrough layer
        self.pth_layer = make_layers(yolov2_cfg[:2], in_channels=512)
        # predict layer
        self.yolo_layer = make_layers(yolov2_cfg[2:], in_channels=1280)
        init_model_variables(self)

    def forward(self, x):
        route, feature = self.darknet19(x)
        route = self.pth_layer(route)
        feature = torch.cat([route, feature], dim=1)
        output = self.yolo_layer(feature)
        output = output.permute(0, 2, 3, 1)
        return output  # 13, 13, 125


In [2]:
ckpt_path = '/home/dk/jyl/Yolo/V2/ckpt/only_params_trained_yolo_coco'

In [45]:
ckpt = torch.load(ckpt_path)
for k, v in ckpt.items():
    print(k + ' --> ' , v.shape)

stage1_conv1.0.weight -->  torch.Size([32, 3, 3, 3])
stage1_conv1.1.weight -->  torch.Size([32])
stage1_conv1.1.bias -->  torch.Size([32])
stage1_conv1.1.running_mean -->  torch.Size([32])
stage1_conv1.1.running_var -->  torch.Size([32])
stage1_conv2.0.weight -->  torch.Size([64, 32, 3, 3])
stage1_conv2.1.weight -->  torch.Size([64])
stage1_conv2.1.bias -->  torch.Size([64])
stage1_conv2.1.running_mean -->  torch.Size([64])
stage1_conv2.1.running_var -->  torch.Size([64])
stage1_conv3.0.weight -->  torch.Size([128, 64, 3, 3])
stage1_conv3.1.weight -->  torch.Size([128])
stage1_conv3.1.bias -->  torch.Size([128])
stage1_conv3.1.running_mean -->  torch.Size([128])
stage1_conv3.1.running_var -->  torch.Size([128])
stage1_conv4.0.weight -->  torch.Size([64, 128, 1, 1])
stage1_conv4.1.weight -->  torch.Size([64])
stage1_conv4.1.bias -->  torch.Size([64])
stage1_conv4.1.running_mean -->  torch.Size([64])
stage1_conv4.1.running_var -->  torch.Size([64])
stage1_conv5.0.weight -->  torch.Size([

In [36]:
yolo = Backbone()

Initialize model's weights ...


In [54]:
for m in yolo.modules():
    if isinstance(m, torch.nn.Conv2d):
        print(m.weight.shape)
        print(m.bias.shape)
#         print(m.)
    if isinstance(m, torch.nn.BatchNorm2d):
        print('BN Bias', m.bias.shape)
        print('BN Weight', m.weight.shape)
        print('BN running mean', m.running_mean.shape)
        print('BN running var', m.running_var.shape)

torch.Size([32, 3, 3, 3])
torch.Size([32])
BN Bias torch.Size([32])
BN Weight torch.Size([32])
BN running mean torch.Size([32])
BN running var torch.Size([32])
torch.Size([64, 32, 3, 3])
torch.Size([64])
BN Bias torch.Size([64])
BN Weight torch.Size([64])
BN running mean torch.Size([64])
BN running var torch.Size([64])
torch.Size([128, 64, 3, 3])
torch.Size([128])
BN Bias torch.Size([128])
BN Weight torch.Size([128])
BN running mean torch.Size([128])
BN running var torch.Size([128])
torch.Size([64, 128, 1, 1])
torch.Size([64])
BN Bias torch.Size([64])
BN Weight torch.Size([64])
BN running mean torch.Size([64])
BN running var torch.Size([64])
torch.Size([128, 64, 3, 3])
torch.Size([128])
BN Bias torch.Size([128])
BN Weight torch.Size([128])
BN running mean torch.Size([128])
BN running var torch.Size([128])
torch.Size([256, 128, 3, 3])
torch.Size([256])
BN Bias torch.Size([256])
BN Weight torch.Size([256])
BN running mean torch.Size([256])
BN running var torch.Size([256])
torch.Size([128